In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@eklavyadatabricks.dfs.core.windows.net/products")

In [0]:
df=df.drop("_rescued_data")
display(df)

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN price*0.90

In [0]:
%sql
SELECT product_id,price, databricks_cata.bronze.discount_func(price) as discounted_price FROM products

In [0]:
df=df.withColumn("discounted_price",expr("databricks_cata.bronze.discount_func(price)"))
display(df)

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
%sql
SELECT product_id, brand, databricks_cata.bronze.upper_func(brand) as brand_upper FROM products

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@eklavyadatabricks.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products
USING DELTA
LOCATION 'abfss://silver@eklavyadatabricks.dfs.core.windows.net/products'